In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from scipy import stats
from sklearn. metrics import roc_curve, auc
import random
from matplotlib.gridspec import GridSpec
from sklearn.calibration import CalibrationDisplay
from sklearn. metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [2]:
def calculate_metrics(y_true, y_pred):
    tp = np.sum(np.logical_and(y_true == 1, y_pred == 1))
    tn = np.sum(np.logical_and(y_true == 0, y_pred == 0))
    fp = np.sum(np.logical_and(y_true == 0, y_pred == 1))
    fn = np.sum(np.logical_and(y_true == 1, y_pred == 0))
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    predictive_accuracy = (tp + tn) / (tp + tn + fp + fn)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return sensitivity, specificity, ppv, npv, predictive_accuracy, mcc

In [4]:
data = pd.read_excel("C:\\Users\\12292\\Desktop\\train size - xx (2).xlsx")

In [5]:
train_X = data.iloc[:, 1:]  # Features (excluding the first column)
train_y = data.iloc[:, 0] 

In [6]:
train_X

,BNF,PHASES,Radscore
0,1.260731,4,0.446201
1,1.385450,9,2.553235
2,1.410795,0,-1.116113
3,0.930106,0,-1.127136
4,1.066182,0,-1.669537
...,...,...,...
328,1.266495,1,-0.748227
329,1.018247,1,-1.270338
330,0.658511,5,0.754189
331,1.007611,6,-0.795665


In [7]:
train_R = data.iloc[:,3]  # Features (excluding the first column)

In [8]:
train_R

0      0.446201
1      2.553235
2     -1.116113
3     -1.127136
4     -1.669537
         ...   
328   -0.748227
329   -1.270338
330    0.754189
331   -0.795665
332    0.604769
Name: Radscore, Length: 333, dtype: float64

In [9]:
train_y 

0      1
1      1
2      0
3      0
4      0
      ..
328    0
329    0
330    1
331    0
332    1
Name: status, Length: 333, dtype: int64

In [10]:
train_P = data.iloc[:,2]  

In [11]:
data_test = pd.read_excel("C:\\Users\\12292\\Desktop\\test size - xx (2).xlsx")

In [12]:
test_y = data_test.iloc[:, 0] 

In [13]:
test_X = data_test.iloc[:, 1:4] 
test_X

,BNF,PHASES,Radscore
0,1.599177,6,-0.199217
1,1.277627,4,0.417314
2,0.932445,1,-0.781816
3,0.956438,5,-0.183729
4,1.548727,3,-0.510796
...,...,...,...
137,1.190633,4,-1.118190
138,1.035633,0,-1.140353
139,1.178697,4,1.206319
140,1.069052,0,-1.283198


In [14]:
test_X

,BNF,PHASES,Radscore
0,1.599177,6,-0.199217
1,1.277627,4,0.417314
2,0.932445,1,-0.781816
3,0.956438,5,-0.183729
4,1.548727,3,-0.510796
...,...,...,...
137,1.190633,4,-1.118190
138,1.035633,0,-1.140353
139,1.178697,4,1.206319
140,1.069052,0,-1.283198


In [15]:
test_R = data_test.iloc[:,3]  

In [16]:
test_y

0      0
1      0
2      0
3      1
4      0
      ..
137    0
138    0
139    1
140    0
141    1
Name: status, Length: 142, dtype: int64

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

rf_params = {
    'n_estimators': 210,
    'random_state':9999,
    'max_depth':3,
    'min_samples_leaf':34,
    'min_samples_split':70,
}

best_RF = RandomForestClassifier(**rf_params)

best_RF.fit(train_X, train_y)
RF_y_pred = best_RF.predict(test_X)
accuracy = accuracy_score(test_y, RF_y_pred)
print("Test Accuracy with Best Parameters:", accuracy)

Test Accuracy with Best Parameters: 0.795774647887324


In [18]:
accuracy = accuracy_score(test_y,  RF_y_pred)
RF_y_score = best_RF.predict_proba(test_X)[:, 1]
fprrf,tprrf,threshold = roc_curve(test_y,RF_y_score)
auc(fprrf,tprrf)

0.8803980099502487

In [19]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve and AUC for the original data
fprrf, tprrf, thresholds = roc_curve(test_y, RF_y_score)
original_auc = auc(fprrf, tprrf)
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this number as needed
auc_scores = []

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):
    # Randomly sample test set samples with replacement
    bootstrap_indices = np.random.randint(0, len(test_y), size=len(test_y))
    bootstrap_y = test_y[bootstrap_indices]
    bootstrap_y_score = RF_y_score[bootstrap_indices]
    
    # Calculate AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
    auc_scores.append(auc_bootstrap)

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)
auc_scores_sorted = np.sort(auc_scores)
n = len(auc_scores)
lower_idx = int(np.round(n * 0.025))  # Calculate lower bound index
upper_idx = int(np.round(n * 0.975))  # Calculate upper bound index
auc_lower = auc_scores_sorted[lower_idx]
auc_upper = auc_scores_sorted[upper_idx]

print("95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))


Original AUC: 0.880
95% CI for AUC: (0.824, 0.941)


In [20]:
from sklearn.metrics import roc_auc_score, roc_curve

# Predict on the training set
RF_train_y_pred = best_RF.predict(train_X)
train_y_prob = best_RF.predict_proba(train_X)[:, 1]

# Calculate AUC
train_auc = roc_auc_score(train_y, train_y_prob)

# Calculate ROC curve and AUC for the original data
fprRF, tprRF, thresholds = roc_curve(train_y, train_y_prob)
original_auc = auc(fprRF, tprRF)
print("Original AUC: {:.3f}".format(original_auc))

Original AUC: 0.916


In [21]:
from sklearn.metrics import roc_auc_score, roc_curve
train_y_prob = best_RF.predict_proba(train_X)[:, 1]

train_auc = roc_auc_score(train_y, train_y_prob)

In [22]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from scipy.stats import percentileofscore

# Calculate original AUC
fprRF, tprRF, thresholds = roc_curve(train_y, train_y_prob)
original_auc = auc(fprRF, tprRF)
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this number as needed
auc_scores = []

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):
    # Randomly sample training set samples with replacement
    bootstrap_indices = np.random.randint(0, len(train_y), size=len(train_y))
    bootstrap_y = train_y[bootstrap_indices]
    bootstrap_y_score = train_y_prob[bootstrap_indices]
    
    # Calculate AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
    auc_scores.append(auc_bootstrap)

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)
auc_scores_sorted = np.sort(auc_scores)
n = len(auc_scores)
lower_idx = int(np.round(n * 0.025))  # Calculate lower bound index
upper_idx = int(np.round(n * 0.975))  # Calculate upper bound index
auc_lower = auc_scores_sorted[lower_idx]
auc_upper = auc_scores_sorted[upper_idx]

print("95% CI for AUC on training set: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))

Original AUC: 0.916
95% CI for AUC on training set: (0.884, 0.943)


In [23]:
# Calculate metrics for the training and test sets
sensitivity_train, specificity_train, ppv_train, npv_train, predictive_accuracy_train, mcc_train = calculate_metrics(train_y, RF_train_y_pred)
sensitivity_test, specificity_test, ppv_test, npv_test, predictive_accuracy_test, mcc_test = calculate_metrics(test_y, RF_y_pred)

# Print results
print("Train Set Metrics:")
print(f"Sensitivity: {sensitivity_train:.3f}")
print(f"Specificity: {specificity_train:.3f}")
print(f"PPV (Precision): {ppv_train:.3f}")
print(f"NPV: {npv_train:.3f}")
print(f"Predictive Accuracy: {predictive_accuracy_train:.3f}")
print(f"MCC: {mcc_train:.3f}")

print("\nTest Set Metrics:")
print(f"Sensitivity: {sensitivity_test:.3f}")
print(f"Specificity: {specificity_test:.3f}")
print(f"PPV (Precision): {ppv_test:.3f}")
print(f"NPV: {npv_test:.3f}")
print(f"Predictive Accuracy: {predictive_accuracy_test:.3f}")
print(f"MCC: {mcc_test:.3f}")

Train Set Metrics:
Sensitivity: 0.823
Specificity: 0.844
PPV (Precision): 0.807
NPV: 0.858
Predictive Accuracy: 0.835
MCC: 0.666

Test Set Metrics:
Sensitivity: 0.746
Specificity: 0.840
PPV (Precision): 0.806
NPV: 0.787
Predictive Accuracy: 0.796
MCC: 0.590


In [24]:
import numpy as np  
from scipy.stats import chi2_contingency  

def hosmer_lemeshow_test(test_y, RF_y_pred, num_groups=10):  
    # Sort the data based on predicted probabilities  
    sorted_indices = np.argsort(RF_y_pred)  
    sorted_y = test_y[sorted_indices]  
    
    # Create groups based on the sorted indices  
    group_size = len(sorted_y) // num_groups  
    group_bounds = np.arange(0, len(sorted_y) + 1, group_size)  
    
    # Calculate observed and expected counts for each group  
    observed_counts = np.zeros(num_groups)  
    expected_counts = np.zeros(num_groups)  
    
    total_positive = np.sum(sorted_y)  
    total_negative = len(sorted_y) - total_positive  
    
    for i in range(num_groups):  
        start, end = group_bounds[i], group_bounds[i + 1]  
        observed_counts[i] = np.sum(sorted_y[start:end])  
        expected_counts[i] = (end - start) * total_positive / len(sorted_y)  
        
    # Perform chi-squared test  
    chi2, p = chi2_contingency(np.array([observed_counts, expected_counts]).T)[:2]  
    
    dof = num_groups - 2  # Degrees of freedom for chi-squared test
      
    return chi2, p, dof  

chi2, p_value, dof = hosmer_lemeshow_test(test_y,RF_y_pred)  
print(f"Chi-squared: {chi2}")  
print(f"P-value: {p_value}")  
print(f"Degrees of freedom: {dof}")

# Determining Model Calibration Based on P-Value
if p_value < 0.05:
    print("The model is poorly calibrated.")
else:
    print("The model is well calibrated.")


Chi-squared: 12.431163258692122
P-value: 0.1900789131095703
Degrees of freedom: 8
The model is well calibrated.


In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
best_model_XGB = xgb.XGBClassifier(n_estimators=254, learning_rate=0.0395, max_depth=3, min_child_weight=7,colsample_bytree=0.4, 
                                   subsample=0.4,random_state=527)

best_model_XGB.fit(train_X, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0395, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              n_estimators=254, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=527, ...)

In [26]:
XGB_train_y_pred = best_model_XGB.predict(train_X)
XGBboost_train_y_score = best_model_XGB.predict_proba(train_X)[:, 1]
fprXGB,tprXGB,threshold = roc_curve(train_y,XGBboost_train_y_score)
auc(fprXGB,tprXGB)

0.9128081340062908

In [27]:
XGB_y_pred = best_model_XGB.predict(test_X)
XGBboost_y_score = best_model_XGB.predict_proba(test_X)[:, 1]
fprxgb,tprxgb,threshold = roc_curve(test_y,XGBboost_y_score)
auc(fprxgb,tprxgb)

0.8877611940298507

In [28]:
from sklearn.metrics import roc_auc_score, roc_curve
train_y_prob =best_model_XGB.predict_proba(train_X)[:, 1]

train_auc = roc_auc_score(train_y, train_y_prob)

In [29]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve and AUC for the original data
fprxgb, tprxgb, thresholds = roc_curve(test_y, XGBboost_y_score )
original_auc = auc(fprxgb, tprxgb)
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this number as needed
auc_scores = []

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):
    # Randomly sample test set samples with replacement
    bootstrap_indices = np.random.randint(0, len(test_y), size=len(test_y))
    bootstrap_y = test_y[bootstrap_indices]
    bootstrap_y_score = XGBboost_y_score[bootstrap_indices]
    
    # Calculate AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
    auc_scores.append(auc_bootstrap)

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)
auc_scores_sorted = np.sort(auc_scores)
n = len(auc_scores)
lower_idx = int(np.round(n * 0.025))  # Calculate lower bound index
upper_idx = int(np.round(n * 0.975))  # Calculate upper bound index
auc_lower = auc_scores_sorted[lower_idx]
auc_upper = auc_scores_sorted[upper_idx]

print("95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))


Original AUC: 0.888
95% CI for AUC: (0.831, 0.946)


In [30]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve and AUC for the original data
fprxgb, tprxgb, thresholds = roc_curve(train_y,XGBboost_train_y_score )
original_auc = auc(fprXGB, tprXGB)
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this number as needed
auc_scores = []

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):
    # Randomly sample train set samples with replacement
    bootstrap_indices = np.random.randint(0, len(train_y), size=len(train_y))
    bootstrap_y = train_y[bootstrap_indices]
    bootstrap_y_score = XGBboost_train_y_score[bootstrap_indices]
    
    # Calculate AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
    auc_scores.append(auc_bootstrap)

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)
auc_scores_sorted = np.sort(auc_scores)
n = len(auc_scores)
lower_idx = int(np.round(n * 0.025))  # Calculate lower bound index
upper_idx = int(np.round(n * 0.975))  # Calculate upper bound index
auc_lower = auc_scores_sorted[lower_idx]
auc_upper = auc_scores_sorted[upper_idx]

print("95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))


Original AUC: 0.913
95% CI for AUC: (0.881, 0.941)


In [31]:
# Metrics
sensitivity_train, specificity_train, ppv_train, npv_train, predictive_accuracy_train, mcc_train = calculate_metrics(train_y,XGB_train_y_pred)
sensitivity_test, specificity_test, ppv_test, npv_test, predictive_accuracy_test, mcc_test = calculate_metrics(test_y,XGB_y_pred)

print("Train Set Metrics:")
print(f"Sensitivity: {sensitivity_train:.4f}")
print(f"Specificity: {specificity_train:.4f}")
print(f"PPV (Precision): {ppv_train:.4f}")
print(f"NPV: {npv_train:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_train:.4f}")
print(f"MCC: {mcc_train:.4f}")

print("\nTest Set Metrics:")
print(f"Sensitivity: {sensitivity_test:.4f}")
print(f"Specificity: {specificity_test:.4f}")
print(f"PPV (Precision): {ppv_test:.3f}")
print(f"NPV: {npv_test:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_test:.4f}")
print(f"MCC: {mcc_test:.4f}")

Train Set Metrics:
Sensitivity: 0.8367
Specificity: 0.8441
PPV (Precision): 0.8092
NPV: 0.8674
Predictive Accuracy: 0.8408
MCC: 0.6787

Test Set Metrics:
Sensitivity: 0.7612
Specificity: 0.8667
PPV (Precision): 0.836
NPV: 0.8025
Predictive Accuracy: 0.8169
MCC: 0.6332


In [32]:
import numpy as np  
from scipy.stats import chi2_contingency  

def hosmer_lemeshow_test(test_y, XGB_y_pred , num_groups=10):  
    # Sort the data based on predicted probabilities  
    sorted_indices = np.argsort(XGB_y_pred)  
    sorted_y = test_y[sorted_indices]  
    
    # Create groups based on the sorted indices  
    group_size = len(sorted_y) // num_groups  
    group_bounds = np.arange(0, len(sorted_y) + 1, group_size)  
    
    # Calculate observed and expected counts for each group  
    observed_counts = np.zeros(num_groups)  
    expected_counts = np.zeros(num_groups)  
    
    total_positive = np.sum(sorted_y)  
    total_negative = len(sorted_y) - total_positive  
    
    for i in range(num_groups):  
        start, end = group_bounds[i], group_bounds[i + 1]  
        observed_counts[i] = np.sum(sorted_y[start:end])  
        expected_counts[i] = (end - start) * total_positive / len(sorted_y)  
        
    # Perform chi-squared test  
    chi2, p = chi2_contingency(np.array([observed_counts, expected_counts]).T)[:2]  
    
    dof = num_groups - 2  # Degrees of freedom for chi-squared test
      
    return chi2, p, dof  

chi2, p_value, dof = hosmer_lemeshow_test(test_y,XGB_y_pred)  
print(f"Chi-squared: {chi2}")  
print(f"P-value: {p_value}")  
print(f"Degrees of freedom: {dof}")

# Determining Model Calibration Based on P-Value
if p_value < 0.05:
    print("The model is poorly calibrated.")
else:
    print("The model is well calibrated.")

Chi-squared: 16.102124381629054
P-value: 0.06477934921791016
Degrees of freedom: 8
The model is well calibrated.


In [33]:
from sklearn.ensemble import GradientBoostingClassifier

In [34]:
    best_GB = GradientBoostingClassifier(learning_rate=0.005,  
                                           n_estimators=278,  
                                           min_samples_split=38,  
                                           min_samples_leaf=27,  
                                           max_depth=3,  
                                           max_features=2,  
                                           subsample=0.7,
                                           random_state=102030)  

    best_GB.fit(train_X, train_y)  

GradientBoostingClassifier(learning_rate=0.005, max_features=2,
                           min_samples_leaf=27, min_samples_split=38,
                           n_estimators=278, random_state=102030,
                           subsample=0.7)

In [35]:
test_gb_pred_proba = best_GB.predict_proba(test_X)[:, 1]
test_auc = roc_auc_score(test_y, test_gb_pred_proba)
GBboost_y_pred=best_GB.predict(test_X)
print("testAUC:", test_auc)
fprgb,tprgb,threshold = roc_curve(test_y,test_gb_pred_proba)
auc(fprgb,tprgb)

testAUC: 0.8906467661691542


0.8906467661691542

In [36]:
fprgb,tprgb,threshold = roc_curve(test_y,test_gb_pred_proba)
auc(fprgb,tprgb)

0.8906467661691542

In [37]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve and AUC for the original data
fprgb, tprgb, thresholds = roc_curve(test_y,test_gb_pred_proba )
original_auc = auc(fprgb, tprgb)
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 10000  # You can adjust this number as needed
auc_scores = []

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):
    # Randomly sample test set samples with replacement
    bootstrap_indices = np.random.randint(0, len(test_y), size=len(test_y))
    bootstrap_y = test_y[bootstrap_indices]
    bootstrap_y_score = test_gb_pred_proba[bootstrap_indices]
    
    # Calculate AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
    auc_scores.append(auc_bootstrap)

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)
auc_scores_sorted = np.sort(auc_scores)
n = len(auc_scores)
lower_idx = int(np.round(n * 0.025))  # Calculate lower bound index
upper_idx = int(np.round(n * 0.975))  # Calculate upper bound index
auc_lower = auc_scores_sorted[lower_idx]
auc_upper = auc_scores_sorted[upper_idx]

print("95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))


Original AUC: 0.891
95% CI for AUC: (0.828, 0.945)


In [38]:
GBboost_train_y_pred=best_GB.predict(train_X)
train_GB_pred_proba = best_GB.predict_proba(train_X)[:, 1]
train_auc = roc_auc_score(train_y, train_GB_pred_proba)
print("Train AUC:", train_auc )

Train AUC: 0.9233230926779313


In [39]:
fprGB,tprGB,threshold = roc_curve(train_y,train_GB_pred_proba)
auc(fprGB,tprGB)

0.9233230926779313

In [40]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve and AUC for the original data
fprGB, tprGB, thresholds = roc_curve(train_y,train_GB_pred_proba)
original_auc = auc(fprGB, tprGB)
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this number as needed
auc_scores = []

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):
    # Randomly sample train set samples with replacement
    bootstrap_indices = np.random.randint(0, len(train_y), size=len(train_y))
    bootstrap_y = train_y[bootstrap_indices]
    bootstrap_y_score = train_GB_pred_proba[bootstrap_indices]
    
    # Calculate AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
    auc_scores.append(auc_bootstrap)

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)
auc_scores_sorted = np.sort(auc_scores)
n = len(auc_scores)
lower_idx = int(np.round(n * 0.025))  # Calculate lower bound index
upper_idx = int(np.round(n * 0.975))  # Calculate upper bound index
auc_lower = auc_scores_sorted[lower_idx]
auc_upper = auc_scores_sorted[upper_idx]

print("95% CI for AUC: ({:.4f}, {:.4f})".format(auc_lower, auc_upper))

Original AUC: 0.923
95% CI for AUC: (0.8951, 0.9483)


In [41]:
# Metrics
sensitivity_train, specificity_train, ppv_train, npv_train, predictive_accuracy_train, mcc_train = calculate_metrics(train_y,GBboost_train_y_pred)
sensitivity_test, specificity_test, ppv_test, npv_test, predictive_accuracy_test, mcc_test = calculate_metrics(test_y,GBboost_y_pred)

print("Train Set Metrics:")
print(f"Sensitivity: {sensitivity_train:.4f}")
print(f"Specificity: {specificity_train:.4f}")
print(f"PPV (Precision): {ppv_train:.4f}")
print(f"NPV: {npv_train:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_train:.4f}")
print(f"MCC: {mcc_train:.4f}")

print("\nTest Set Metrics:")
print(f"Sensitivity: {sensitivity_test:.4f}")
print(f"Specificity: {specificity_test:.4f}")
print(f"PPV (Precision): {ppv_test:.3f}")
print(f"NPV: {npv_test:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_test:.4f}")
print(f"MCC: {mcc_test:.4f}")

Train Set Metrics:
Sensitivity: 0.8367
Specificity: 0.8495
PPV (Precision): 0.8146
NPV: 0.8681
Predictive Accuracy: 0.8438
MCC: 0.6844

Test Set Metrics:
Sensitivity: 0.7463
Specificity: 0.8533
PPV (Precision): 0.820
NPV: 0.7901
Predictive Accuracy: 0.8028
MCC: 0.6047


In [42]:
best_GB_y_score =best_GB.predict_proba(test_X)[:, 1]

In [43]:
import numpy as np  
from scipy.stats import chi2_contingency  

def hosmer_lemeshow_test(test_y, GBboost_y_pred , num_groups=10):  
    # Sort the data based on predicted probabilities  
    sorted_indices = np.argsort( GBboost_y_pred )  
    sorted_y = test_y[sorted_indices]  
    
    # Create groups based on the sorted indices  
    group_size = len(sorted_y) // num_groups  
    group_bounds = np.arange(0, len(sorted_y) + 1, group_size)  
    
    # Calculate observed and expected counts for each group  
    observed_counts = np.zeros(num_groups)  
    expected_counts = np.zeros(num_groups)  
    
    total_positive = np.sum(sorted_y)  
    total_negative = len(sorted_y) - total_positive  
    
    for i in range(num_groups):  
        start, end = group_bounds[i], group_bounds[i + 1]  
        observed_counts[i] = np.sum(sorted_y[start:end])  
        expected_counts[i] = (end - start) * total_positive / len(sorted_y)  
        
    # Perform chi-squared test  
    chi2, p = chi2_contingency(np.array([observed_counts, expected_counts]).T)[:2]  
    
    dof = num_groups - 2  # Degrees of freedom for chi-squared test
      
    return chi2, p, dof  

chi2, p_value, dof = hosmer_lemeshow_test(test_y, GBboost_y_pred )  
print(f"Chi-squared: {chi2}")  
print(f"P-value: {p_value}")  
print(f"Degrees of freedom: {dof}")


# Determining Model Calibration Based on P-Value
if p_value < 0.05:
    print("The model is poorly calibrated.")
else:
    print("The model is well calibrated.")

Chi-squared: 13.931517965112386
P-value: 0.12478220479239868
Degrees of freedom: 8
The model is well calibrated.


In [44]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV


best_params_LGBM = {
    'lambda_l1': 0.7,
    'lambda_l2': 1e-05,
    'learning_rate': 0.005,
    'n_estimators': 600,
    'max_depth': 4,
    'num_leaves': 12,
    'max_bin': 14,
    'min_data_in_leaf': 32,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.6,
    'bagging_freq': 10,
    'random_state': 123,
    'verbose': -1
}


best_LGBM = lgb.LGBMClassifier(**best_params_LGBM)
best_LGBM.fit(train_X, train_y)

LGBMClassifier(bagging_fraction=0.6, bagging_freq=10, feature_fraction=0.9,
               lambda_l1=0.7, lambda_l2=1e-05, learning_rate=0.005, max_bin=14,
               max_depth=4, min_data_in_leaf=32, n_estimators=600,
               num_leaves=12, random_state=123, verbose=-1)

In [45]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
train_y_pred = best_LGBM.predict(train_X)
train_accuracy = accuracy_score(train_y, train_y_pred)
print("Training Accuracy:", train_accuracy)
train_y_lgbm_prob = best_LGBM.predict_proba(train_X)[:, 1]
train_auc = roc_auc_score(train_y, train_y_prob)
print("Training AUC：", train_auc)

Training Accuracy: 0.8378378378378378
Training AUC： 0.9128081340062908


In [46]:
y_score_LGBM = best_LGBM.predict_proba(test_X)[:, 1]
LGBM_y_pred=best_LGBM.predict(test_X)
test_accuracy = accuracy_score(test_y, LGBM_y_pred)
print("Test_accuracy:", test_accuracy)
fprlgbm, tprlgbm, threshold = roc_curve(test_y, y_score_LGBM)
test_auc = roc_auc_score(test_y, y_score_LGBM )

print("Test AUC:", test_auc)

Test_accuracy: 0.823943661971831
Test AUC: 0.8920398009950249


In [47]:
# Metrics
sensitivity_train, specificity_train, ppv_train, npv_train, predictive_accuracy_train, mcc_train = calculate_metrics(train_y, train_y_pred)
sensitivity_test, specificity_test, ppv_test, npv_test, predictive_accuracy_test, mcc_test = calculate_metrics(test_y, LGBM_y_pred)

print("Train Set Metrics:")
print(f"Sensitivity: {sensitivity_train:.4f}")
print(f"Specificity: {specificity_train:.4f}")
print(f"PPV (Precision): {ppv_train:.4f}")
print(f"NPV: {npv_train:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_train:.4f}")
print(f"MCC: {mcc_train:.4f}")

print("\nTest Set Metrics:")
print(f"Sensitivity: {sensitivity_test:.4f}")
print(f"Specificity: {specificity_test:.4f}")
print(f"PPV (Precision): {ppv_test:.3f}")
print(f"NPV: {npv_test:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_test:.4f}")
print(f"MCC: {mcc_test:.4f}")

Train Set Metrics:
Sensitivity: 0.8231
Specificity: 0.8495
PPV (Precision): 0.8121
NPV: 0.8587
Predictive Accuracy: 0.8378
MCC: 0.6717

Test Set Metrics:
Sensitivity: 0.7761
Specificity: 0.8667
PPV (Precision): 0.839
NPV: 0.8125
Predictive Accuracy: 0.8239
MCC: 0.6470


In [48]:
fprLGBM, tprLGBM, thresholds= roc_curve(train_y, train_y_lgbm_prob)
fprlgbm, tprlgbm, thresholds = roc_curve(test_y, y_score_LGBM)

In [49]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve and AUC for the original data
fprlgbm, tprlgbm, thresholds = roc_curve(test_y,y_score_LGBM)
original_auc = auc(fprlgbm, tprlgbm)
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 10000  # You can adjust this number as needed
auc_scores = []

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):
    # Randomly sample test set samples with replacement
    bootstrap_indices = np.random.randint(0, len(test_y), size=len(test_y))
    bootstrap_y = test_y[bootstrap_indices]
    bootstrap_y_score = y_score_LGBM [bootstrap_indices]
    
    # Calculate AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
    auc_scores.append(auc_bootstrap)

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)
auc_scores_sorted = np.sort(auc_scores)
n = len(auc_scores)
lower_idx = int(np.round(n * 0.025))  # Calculate lower bound index
upper_idx = int(np.round(n * 0.975))  # Calculate upper bound index
auc_lower = auc_scores_sorted[lower_idx]
auc_upper = auc_scores_sorted[upper_idx]

print("95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))


Original AUC: 0.892
95% CI for AUC: (0.830, 0.946)


In [50]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve and AUC for the original data
fprLGBM, tprLGBM, thresholds = roc_curve(train_y,train_y_lgbm_prob )
original_auc = auc(fprLGBM, tprLGBM)
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this number as needed
auc_scores = []

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):
    # Randomly sample train set samples with replacement
    bootstrap_indices = np.random.randint(0, len(train_y), size=len(train_y))
    bootstrap_y = train_y[bootstrap_indices]
    bootstrap_y_score = train_y_lgbm_prob[bootstrap_indices]
    
    # Calculate AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
    auc_scores.append(auc_bootstrap)

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)
auc_scores_sorted = np.sort(auc_scores)
n = len(auc_scores)
lower_idx = int(np.round(n * 0.025))  # Calculate lower bound index
upper_idx = int(np.round(n * 0.975))  # Calculate upper bound index
auc_lower = auc_scores_sorted[lower_idx]
auc_upper = auc_scores_sorted[upper_idx]

print("95% CI for AUC: ({:.4f}, {:.4f})".format(auc_lower, auc_upper))


Original AUC: 0.917
95% CI for AUC: (0.8871, 0.9442)


In [51]:
import numpy as np  
from scipy.stats import chi2_contingency  

def hosmer_lemeshow_test(test_y, LGBM_y_pred , num_groups=10):  
    # Sort the data based on predicted probabilities  
    sorted_indices = np.argsort( LGBM_y_pred)  
    sorted_y = test_y[sorted_indices]  
    
    # Create groups based on the sorted indices  
    group_size = len(sorted_y) // num_groups  
    group_bounds = np.arange(0, len(sorted_y) + 1, group_size)  
    
    # Calculate observed and expected counts for each group  
    observed_counts = np.zeros(num_groups)  
    expected_counts = np.zeros(num_groups)  
    
    total_positive = np.sum(sorted_y)  
    total_negative = len(sorted_y) - total_positive  
    
    for i in range(num_groups):  
        start, end = group_bounds[i], group_bounds[i + 1]  
        observed_counts[i] = np.sum(sorted_y[start:end])  
        expected_counts[i] = (end - start) * total_positive / len(sorted_y)  
        
    # Perform chi-squared test  
    chi2, p = chi2_contingency(np.array([observed_counts, expected_counts]).T)[:2]  
    
    dof = num_groups - 2  # Degrees of freedom for chi-squared test
      
    return chi2, p, dof  

chi2, p_value, dof = hosmer_lemeshow_test(test_y,LGBM_y_pred)  
print(f"Chi-squared: {chi2}")  
print(f"P-value: {p_value}")  
print(f"Degrees of freedom: {dof}")

# Determining Model Calibration Based on P-Value
if p_value < 0.05:
    print("The model is poorly calibrated.")
else:
    print("The model is well calibrated.")

Chi-squared: 14.752584657314266
P-value: 0.09796011603922684
Degrees of freedom: 8
The model is well calibrated.


In [52]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as st
from sklearn import metrics

class DelongTest():
    def __init__(self, preds1, preds2, label, threshold=0.05):
        self._preds1 = preds1
        self._preds2 = preds2
        self._label = label
        self.threshold = threshold
        self._show_result()

    def _auc(self, X, Y) -> float:
        return 1/(len(X)*len(Y)) * sum([self._kernel(x, y) for x in X for y in Y])

    def _kernel(self, X, Y) -> float:
        return .5 if Y == X else int(Y < X)

    def _structural_components(self, X, Y) -> list:
        V10 = [1/len(Y) * sum([self._kernel(x, y) for y in Y]) for x in X]
        V01 = [1/len(X) * sum([self._kernel(x, y) for x in X]) for y in Y]
        return V10, V01

    def _get_S_entry(self, V_A, V_B, auc_A, auc_B) -> float:
        return 1/(len(V_A)-1) * sum([(a-auc_A)*(b-auc_B) for a, b in zip(V_A, V_B)])
    
    def _z_score(self, var_A, var_B, covar_AB, auc_A, auc_B):
        return (auc_A - auc_B) / ((var_A + var_B - 2*covar_AB)**(.5) + 1e-8)

    def _group_preds_by_label(self, preds, actual) -> list:
        X = [p for (p, a) in zip(preds, actual) if a]
        Y = [p for (p, a) in zip(preds, actual) if not a]
        return X, Y

    def _compute_z_p(self):
        X_A, Y_A = self._group_preds_by_label(self._preds1, self._label)
        X_B, Y_B = self._group_preds_by_label(self._preds2, self._label)

        V_A10, V_A01 = self._structural_components(X_A, Y_A)
        V_B10, V_B01 = self._structural_components(X_B, Y_B)

        auc_A = self._auc(X_A, Y_A)
        auc_B = self._auc(X_B, Y_B)

        var_A = (self._get_S_entry(V_A10, V_A10, auc_A, auc_A) * 1/len(V_A10) + self._get_S_entry(V_A01, V_A01, auc_A, auc_A) * 1/len(V_A01))
        var_B = (self._get_S_entry(V_B10, V_B10, auc_B, auc_B) * 1/len(V_B10) + self._get_S_entry(V_B01, V_B01, auc_B, auc_B) * 1/len(V_B01))
        covar_AB = (self._get_S_entry(V_A10, V_B10, auc_A, auc_B) * 1/len(V_A10) + self._get_S_entry(V_A01, V_B01, auc_A, auc_B) * 1/len(V_A01))

        z = self._z_score(var_A, var_B, covar_AB, auc_A, auc_B)
        p = st.norm.sf(abs(z)) * 2

        return z, p

    def _show_result(self):
        z, p = self._compute_z_p()
        print(f"z score = {z:.5f};\np value = {p:.5f};")
        if p < self.threshold:
            print("There is a significant difference")
        else:
            print("There is NO significant difference")

# Assuming you have defined predictions and model_names appropriately
predictions = [y_score_LGBM, test_gb_pred_proba, XGBboost_y_score, RF_y_score]
model_names = ["LGBM", "GB", "XGB", "RF"]

# Compare each pair of models
for i in range(len(predictions)):
    for j in range(i+1, len(predictions)):
        model1_preds = predictions[i]
        model2_preds = predictions[j]
        model1_name = model_names[i]
        model2_name = model_names[j]
        
        # Use the DelongTest class to compare predictions between two models
        delong_test = DelongTest(model1_preds, model2_preds, test_y)
        print(f"Comparison between {model1_name} and {model2_name}:")
        delong_test._show_result()
        print("\n")

z score = 0.15964;
p value = 0.87316;
There is NO significant difference
Comparison between LGBM and GB:
z score = 0.15964;
p value = 0.87316;
There is NO significant difference


z score = 0.60502;
p value = 0.54516;
There is NO significant difference
Comparison between LGBM and XGB:
z score = 0.60502;
p value = 0.54516;
There is NO significant difference


z score = 1.43431;
p value = 0.15148;
There is NO significant difference
Comparison between LGBM and RF:
z score = 1.43431;
p value = 0.15148;
There is NO significant difference


z score = 0.40568;
p value = 0.68498;
There is NO significant difference
Comparison between GB and XGB:
z score = 0.40568;
p value = 0.68498;
There is NO significant difference


z score = 1.89462;
p value = 0.05814;
There is NO significant difference
Comparison between GB and RF:
z score = 1.89462;
p value = 0.05814;
There is NO significant difference


z score = 1.40008;
p value = 0.16149;
There is NO significant difference
Comparison between XGB and RF

In [55]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as st
from sklearn import metrics

class DelongTest():
    def __init__(self, preds1, preds2, label, threshold=0.05):
        self._preds1 = preds1
        self._preds2 = preds2
        self._label = label
        self.threshold = threshold
        self._show_result()

    def _auc(self, X, Y) -> float:
        return 1/(len(X)*len(Y)) * sum([self._kernel(x, y) for x in X for y in Y])

    def _kernel(self, X, Y) -> float:
        return .5 if Y == X else int(Y < X)

    def _structural_components(self, X, Y) -> list:
        V10 = [1/len(Y) * sum([self._kernel(x, y) for y in Y]) for x in X]
        V01 = [1/len(X) * sum([self._kernel(x, y) for x in X]) for y in Y]
        return V10, V01

    def _get_S_entry(self, V_A, V_B, auc_A, auc_B) -> float:
        return 1/(len(V_A)-1) * sum([(a-auc_A)*(b-auc_B) for a, b in zip(V_A, V_B)])
    
    def _z_score(self, var_A, var_B, covar_AB, auc_A, auc_B):
        return (auc_A - auc_B) / ((var_A + var_B - 2*covar_AB)**(.5) + 1e-8)

    def _group_preds_by_label(self, preds, actual) -> list:
        X = [p for (p, a) in zip(preds, actual) if a]
        Y = [p for (p, a) in zip(preds, actual) if not a]
        return X, Y

    def _compute_z_p(self):
        X_A, Y_A = self._group_preds_by_label(self._preds1, self._label)
        X_B, Y_B = self._group_preds_by_label(self._preds2, self._label)

        V_A10, V_A01 = self._structural_components(X_A, Y_A)
        V_B10, V_B01 = self._structural_components(X_B, Y_B)

        auc_A = self._auc(X_A, Y_A)
        auc_B = self._auc(X_B, Y_B)

        var_A = (self._get_S_entry(V_A10, V_A10, auc_A, auc_A) * 1/len(V_A10) + self._get_S_entry(V_A01, V_A01, auc_A, auc_A) * 1/len(V_A01))
        var_B = (self._get_S_entry(V_B10, V_B10, auc_B, auc_B) * 1/len(V_B10) + self._get_S_entry(V_B01, V_B01, auc_B, auc_B) * 1/len(V_B01))
        covar_AB = (self._get_S_entry(V_A10, V_B10, auc_A, auc_B) * 1/len(V_A10) + self._get_S_entry(V_A01, V_B01, auc_A, auc_B) * 1/len(V_A01))

        z = self._z_score(var_A, var_B, covar_AB, auc_A, auc_B)
        p = st.norm.sf(abs(z)) * 2

        return z, p

    def _show_result(self):
        z, p = self._compute_z_p()
        print(f"z score = {z:.5f};\np value = {p:.5f};")
        if p < self.threshold:
            print("There is a significant difference")
        else:
            print("There is NO significant difference")

# Define your best models and training set predictions
train_y_lgbm_prob = best_LGBM.predict_proba(train_X)[:, 1]
train_gb_pred_proba = best_GB.predict_proba(train_X)[:, 1]
train_xgb_pred_proba = best_model_XGB.predict_proba(train_X)[:, 1]
train_rf_pred_proba = best_RF.predict_proba(train_X)[:, 1]

# Calculate AUCs for the training set
train_auc_lgbm = roc_auc_score(train_y, train_y_lgbm_prob)
train_auc_gb = roc_auc_score(train_y, train_gb_pred_proba)
train_auc_xgb = roc_auc_score(train_y, train_xgb_pred_proba)
train_auc_rf = roc_auc_score(train_y, train_rf_pred_proba)

print(f"Training AUCs:\nLGBM: {train_auc_lgbm}\nGB: {train_auc_gb}\nXGB: {train_auc_xgb}\nRF: {train_auc_rf}")

# Assuming you have defined predictions and model_names appropriately for the training set
train_predictions = [train_y_lgbm_prob, train_gb_pred_proba, train_xgb_pred_proba, train_rf_pred_proba]
train_model_names = ["LGBM", "GB", "XGB", "RF"]

# Compare each pair of models on the training set
for i in range(len(train_predictions)):
    for j in range(i+1, len(train_predictions)):
        model1_preds = train_predictions[i]
        model2_preds = train_predictions[j]
        model1_name = train_model_names[i]
        model2_name = train_model_names[j]
        
        # Use the DelongTest class to compare predictions between two models
        delong_test = DelongTest(model1_preds, model2_preds, train_y)
        print(f"Comparison between {model1_name} and {model2_name} on the training set:")
        delong_test._show_result()
        print("\n")


Training AUCs:
LGBM: 0.9172701338599956
GB: 0.9233230926779313
XGB: 0.9128081340062908
RF: 0.915697461780411
z score = -1.77603;
p value = 0.07573;
There is NO significant difference
Comparison between LGBM and GB on the training set:
z score = -1.77603;
p value = 0.07573;
There is NO significant difference


z score = 1.20354;
p value = 0.22877;
There is NO significant difference
Comparison between LGBM and XGB on the training set:
z score = 1.20354;
p value = 0.22877;
There is NO significant difference


z score = 0.48530;
p value = 0.62747;
There is NO significant difference
Comparison between LGBM and RF on the training set:
z score = 0.48530;
p value = 0.62747;
There is NO significant difference


z score = 2.65714;
p value = 0.00788;
There is a significant difference
Comparison between GB and XGB on the training set:
z score = 2.65714;
p value = 0.00788;
There is a significant difference


z score = 2.48896;
p value = 0.01281;
There is a significant difference
Comparison between 

In [53]:
import pandas as pd

class DelongTest():
    def __init__(self, preds1, preds2, label, model1_name, model2_name, threshold=0.05):
        self._preds1 = preds1
        self._preds2 = preds2
        self._label = label
        self.model1_name = model1_name
        self.model2_name = model2_name
        self.threshold = threshold
        self.results = self._compute_result()

    def _auc(self, X, Y) -> float:
        return 1 / (len(X) * len(Y)) * sum([self._kernel(x, y) for x in X for y in Y])

    def _kernel(self, X, Y) -> float:
        return 0.5 if Y == X else int(Y < X)

    def _structural_components(self, X, Y) -> list:
        V10 = [1 / len(Y) * sum([self._kernel(x, y) for y in Y]) for x in X]
        V01 = [1 / len(X) * sum([self._kernel(x, y) for x in X]) for y in Y]
        return V10, V01

    def _get_S_entry(self, V_A, V_B, auc_A, auc_B) -> float:
        return 1 / (len(V_A) - 1) * sum([(a - auc_A) * (b - auc_B) for a, b in zip(V_A, V_B)])

    def _z_score(self, var_A, var_B, covar_AB, auc_A, auc_B):
        return (auc_A - auc_B) / ((var_A + var_B - 2 * covar_AB) ** (.5) + 1e-8)

    def _group_preds_by_label(self, preds, actual) -> list:
        X = [p for (p, a) in zip(preds, actual) if a]
        Y = [p for (p, a) in zip(preds, actual) if not a]
        return X, Y

    def _compute_result(self):
        X_A, Y_A = self._group_preds_by_label(self._preds1, self._label)
        X_B, Y_B = self._group_preds_by_label(self._preds2, self._label)

        V_A10, V_A01 = self._structural_components(X_A, Y_A)
        V_B10, V_B01 = self._structural_components(X_B, Y_B)

        auc_A = self._auc(X_A, Y_A)
        auc_B = self._auc(X_B, Y_B)

        var_A = (self._get_S_entry(V_A10, V_A10, auc_A, auc_A) * 1 / len(V_A10) + self._get_S_entry(V_A01, V_A01,
                                                                                                      auc_A,
                                                                                                      auc_A) * 1 / len(
            V_A01))
        var_B = (self._get_S_entry(V_B10, V_B10, auc_B, auc_B) * 1 / len(V_B10) + self._get_S_entry(V_B01, V_B01,
                                                                                                      auc_B,
                                                                                                      auc_B) * 1 / len(
            V_B01))
        covar_AB = (self._get_S_entry(V_A10, V_B10, auc_A, auc_B) * 1 / len(V_A10) + self._get_S_entry(V_A01,
                                                                                                       V_B01, auc_A,
                                                                                                       auc_B) * 1 / len(
            V_A01))

        z = self._z_score(var_A, var_B, covar_AB, auc_A, auc_B)
        p = st.norm.sf(abs(z)) * 2

        result = {
            "Model 1": self.model1_name,
            "Model 2": self.model2_name,
            "z score": z,
            "p value": p,
            "Significant Difference": "Yes" if p < self.threshold else "No"
        }
        return result


# Create an empty list to store the results
results_list = []

for i in range(len(predictions)):
    for j in range(i + 1, len(predictions)):
        model1_preds = predictions[i]
        model2_preds = predictions[j]
        model1_name = model_names[i]
        model2_name = model_names[j]

        delong_test = DelongTest(model1_preds, model2_preds, test_y, model1_name, model2_name)
        results_list.append(delong_test.results)

# Convert the list of results into a DataFrame
results_df = pd.DataFrame(results_list)

# Save the DataFrame as a CSV file
#results_df.to_csv("model_comparison_results1.csv", index=False)

In [54]:
from sklearn.metrics import brier_score_loss

# Brier 
LGBM_brier_score = brier_score_loss(test_y, y_score_LGBM)
GB_brier_score = brier_score_loss(test_y, test_gb_pred_proba)
XGB_brier_score = brier_score_loss(test_y, XGBboost_y_score)
RF_brier_score = brier_score_loss(test_y, RF_y_score)

print("LGBM Brier score:", LGBM_brier_score)
print("GB Brier score:", GB_brier_score)
print("XGB Brier score:", XGB_brier_score)
print("RF Brier score:", RF_brier_score)

LGBM Brier score: 0.1320138283556927
GB Brier score: 0.1427594910326072
XGB Brier score: 0.13496849132066688
RF Brier score: 0.14742283265438752
